# **MBD - Websites malicious or not clasification**

# Docker Setup

In [ ]:
!pip install -q sklearn
!pip install seaborn
!pip install pandas

In [ ]:
docker run -it -p 8888:8888 tensorflow/tensorflow:latest-gpu-jupyter jupyter notebook --notebook-dir=/tf --ip 0.0.0.0 --no-browser --allow-root --NotebookApp.allow_origin='https://colab.research.google.com'

# DATA upload 

In [ ]:
import pandas as pd
import re

In [ ]:
def loadDataset(file_name):
 df = pd.read_csv(file_name)
 return df

In [ ]:
df = loadDataset("FinalPersonalAndBigData.csv")

In [ ]:
df.columns

In [ ]:
df = df[['geo_loc_id', 'tld_id', 'url_len', 'who_is', 'https', 'bad','abnormal_url', 'count_dot', 'count-www', 'count@', 'count_dir', 'count_embed_domain', 'short_url', 'count-http', 'africa', 'asia','europe', 'n_america', 's_america', 'oceania']]

## Data good-bad normalize

In [ ]:
badCount = df[df.bad == 1]
len(badCount)

In [ ]:
goodCount = df[df.bad == 0]
len(goodCount)

In [ ]:
goodOnes = goodCount.head(30000)

In [ ]:
df = pd.concat([goodOnes, badCount])

# Dataset details

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
#SHUFFLE dataset
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

# Text to 0 or 1

In [ ]:
def bad_or_not(label):
    if label == 1:
        return 0
    else:
        return 1
df['bad'] = df['bad'].apply(lambda i: bad_or_not(i))
df.head()

In [ ]:
def who_is_or_not(who_is):
    match = re.search('incomplete', who_is)
    if match:
        return 0
    else:
        return 1
df['who_is'] = df['who_is'].apply(lambda i: who_is_or_not(i))
df.head()

In [ ]:
def https_or_not(https):
    match = re.search('yes', https)
    if match:
        return 1
    else:
        return 0
df['https'] = df['https'].apply(lambda i: https_or_not(i))
df.head()

# Adding more identifiers

In [ ]:
def africa_country(country):
  match = re.search('Algeria|Angola|Benin|Botswana|Burkina|Burundi|Cameroon|Cape Verde|Central African Republic|Chad|Comoros|Congo|Congo, Democratic Republic of|Djibouti|Egypt|Equatorial Guinea|Eritrea|Ethiopia|Gabon|Gambia|Ghana|Guinea|Guinea-Bissau|Ivory Coast|Kenya|Lesotho|Liberia|Libya|Madagascar|Malawi|Mali|Mauritania|Mauritius|Morocco|Mozambique|Namibia|Niger|Nigeria|Rwanda|Sao Tome and Principe|Senegal|Seychelles|Sierra Leone|Somalia|South Africa|South Sudan|Sudan|Swaziland|Tanzania|Togo|Tunisia|Uganda|Zambia|Zimbabwe', country)
  if match:
        return 1
  else:
        return 0

def asia_country(country):
  match = re.search('Afghanistan|Bahrain|Bangladesh|Bhutan|Brunei|Burma (Myanmar)|Cambodia|China|East Timor|India|Indonesia|Iran|Iraq|Israel|Japan|Jordan|Kazakhstan|Korea, North|Korea, South|Kuwait|Kyrgyzstan|Laos|Lebanon|Malaysia|Maldives|Mongolia|Nepal|Oman|Pakistan|Philippines|Qatar|Russian Federation|Saudi Arabia|Singapore|Sri Lanka|Syria|Tajikistan|Thailand|Turkey|Turkmenistan|United Arab Emirates|Uzbekistan|Vietnam|Yemen', country)
  if match:
        return 1
  else:
        return 0

def europe_country(country):
  match = re.search('Albania|Andorra|Armenia|Austria|Azerbaijan|Belarus|Belgium|Bosnia and Herzegovina|Bulgaria|Croatia|Cyprus|CZ|Denmark|Estonia|Finland|France|Georgia|Germany|Greece|Hungary|Iceland|Ireland|Italy|Latvia|Liechtenstein|Lithuania|Luxembourg|Macedonia|Malta|Moldova|Monaco|Montenegro|Netherlands|Norway|Poland|Portugal|Romania|San Marino|Serbia|Slovakia|Slovenia|Spain|Sweden|Switzerland|Ukraine|United Kingdom|Vatican City', country)
  if match:
        return 1
  else:
        return 0

def n_america_country(country):
  match = re.search('Antigua and Barbuda|Bahamas|Barbados|Belize|Canada|Costa Rica|Cuba|Dominica|Dominican Republic|El Salvador|Grenada|Guatemala|Haiti|Honduras|Jamaica|Mexico|Nicaragua|Panama|Saint Kitts and Nevis|Saint Lucia|Saint Vincent and the Grenadines|Trinidad and Tobago|US|United States', country)
  if match:
        return 1
  else:
        return 0

def s_america_country(country):
  match = re.search('Argentina|Bolivia|Brazil|Chile|Colombia|Ecuador|Guyana|Paraguay|Peru|Suriname|Uruguay|Venezuela', country)
  if match:
        return 1
  else:
        return 0

def oceania_country(country):
  match = re.search('Australia|Fiji|Kiribati|Marshall Islands|Micronesia|Nauru|New Zealand|Palau|Papua New Guinea|Samoa|Solomon Islands|Taiwan|Tonga|Tuvalu|Vanuatu', country)
  if match:
        return 1
  else:
        return 0

df["africa"] = df["geo_loc"].apply(lambda i: africa_country(i))
df["asia"] = df["geo_loc"].apply(lambda i: asia_country(i))
df["europe"] = df["geo_loc"].apply(lambda i: europe_country(i))
df["n_america"] = df["geo_loc"].apply(lambda i: n_america_country(i))
df["s_america"] = df["geo_loc"].apply(lambda i: s_america_country(i))
df["oceania"] = df["geo_loc"].apply(lambda i: oceania_country(i))


In [ ]:
df = df.drop(columns=["geo_loc"])

In [ ]:
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        return 1
    else:
        return 0
df['use_of_ip'] = df['url'].apply(lambda i: having_ip_address(i))
df.groupby('use_of_ip').size()

In [ ]:
df.groupby('use_of_ip').size()

In [ ]:
from urllib.parse import urlparse

def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        return 1
    else:
        return 0

df['abnormal_url'] = df['url'].apply(lambda i: abnormal_url(i))

df.groupby('abnormal_url').size()

In [ ]:
df['count_dot'] = df['url'].apply(lambda i: i.count('.'))
df.groupby('count_dot').size()

In [ ]:
df['count-www'] = df['url'].apply(lambda i: i.count('www'))
df['count@'] = df['url'].apply(lambda i: i.count('@'))
from urllib.parse import urlparse
def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')
df['count_dir'] = df['url'].apply(lambda i: no_of_dir(i))
def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')
df['count_embed_domain'] = df['url'].apply(lambda i: no_of_embed(i))
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0
df['short_url'] = df['url'].apply(lambda i: shortening_service(i))

df.groupby('count-www').size()
df.groupby('count@').size()
df.groupby('count_dir').size()
df.groupby('count_embed_domain').size()

In [ ]:
df['count-https'] = df['url'].apply(lambda i : i.count('https'))
#df['count-http'] = df['url'].apply(lambda i : i.count('http'))

df.groupby('count-https').size()
#df.groupby('count-http').size()

In [ ]:
# df.to_csv("FinalPersonalAndBigData.csv")

# Data to ids

GEO LOC to ids:

In [ ]:
df.groupby("geo_loc").size().sort_values()

In [ ]:
df["geo_loc"] = df["geo_loc"].astype('category')
df["geo_loc_id"] = df["geo_loc"].cat.codes
df.head()

In [ ]:
df["tld"] = df["tld"].astype('category')
df["tld_id"] = df["tld"].cat.codes
df.head(10).T

In [ ]:
id_to_country = pd.Series(df.geo_loc.values,index=df.geo_loc_id).to_dict()
f = open("id_to_country.js", "a")
f.truncate(0)
f.write("const id_to_country = " + str(id_to_country) + "\n" + "module.exports = id_to_country;")
f.close()

In [ ]:
id_to_tld = pd.Series(df.tld.values,index=df.tld_id).to_dict()
f = open("id_to_tld.js", "a")
f.write("const id_to_tld = " + str(id_to_tld) + "\n" + "module.exports = id_to_tld;")
f.close()

# DATA visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.hist(bins=50, figsize=(20, 20))

In [ ]:
df.groupby('tld_id').bad.mean().plot(kind='barh').set_xlabel('% bad')

In [ ]:
plt.figure(figsize=(15,13))
sns.heatmap(df.corr())
plt.show()

In [ ]:
columns = x_train.columns
print(columns)

In [ ]:
import matplotlib.pyplot as plt

def plot(feature, x=None, y=None):
    plt.figure(figsize=(10, 8))
    plt.scatter(x_train[feature], y_train, label='Data')
    if x is not None and y is not None:
        plt.plot(x, y, color='k', label='Predictions')
    plt.xlabel(feature)
    plt.ylabel('bad')
    plt.legend()

In [ ]:
plot("geo_loc_id")

In [ ]:
sns.set(rc={'figure.figsize':(20,20)})
sns.heatmap(df.corr(), annot=True)

# Separate x and y

In [ ]:
x = df.drop(columns=["bad"])
x

In [ ]:
y = df["bad"]
y

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Machine Learning methods

*   Decission Tree
*   Random Forrest Classifier
*   Multilayer Perceptrons

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
ML_Model = []
acc_train = []
acc_test = []

def storeResults(model, a,b):
  ML_Model.append(model)
  acc_train.append(round(a, 3))
  acc_test.append(round(b, 3))

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth = 5)
tree.fit(x_train, y_train)

In [ ]:
y_test_tree = tree.predict(x_test)
y_train_tree = tree.predict(x_train)

## Evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
acc_train_tree = accuracy_score(y_train,y_train_tree)
acc_test_tree = accuracy_score(y_test,y_test_tree)

print("Decision Tree: Accuracy on training Data: {:.3f}".format(acc_train_tree))
print("Decision Tree: Accuracy on test Data: {:.3f}".format(acc_test_tree))

In [ ]:
plt.figure(figsize=(9,7))
n_features = x_train.shape[1]
plt.barh(range(n_features), tree.feature_importances_, align='center')
plt.yticks(np.arange(n_features), x_train.columns)
plt.xlabel("Savybės svarbumas")
plt.ylabel("Savybė")
plt.show()

## Save results

In [ ]:
storeResults('Decision Tree', acc_train_tree, acc_test_tree)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(max_depth=5)

forest.fit(x_train, y_train)

In [ ]:
y_test_forest = forest.predict(x_test)
y_train_forest = forest.predict(x_train)

## Evaluation

In [ ]:
acc_train_forest = accuracy_score(y_train,y_train_forest)
acc_test_forest = accuracy_score(y_test,y_test_forest)

print("Random forest: Accuracy on training Data: {:.3f}".format(acc_train_forest))
print("Random forest: Accuracy on test Data: {:.3f}".format(acc_test_forest))

In [ ]:
plt.figure(figsize=(9,7))
n_features = x_train.shape[1]
plt.barh(range(n_features), forest.feature_importances_, align='center')
plt.yticks(np.arange(n_features), x_train.columns)
plt.xlabel("Savybės svarbumas")
plt.ylabel("Savybė")
plt.show()

In [ ]:
storeResults('Random Forest', acc_train_forest, acc_test_forest)

# Multilayer Perceptrons (MLPs): Deep Learning

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=0.001, hidden_layer_sizes=([100,100,100]))

mlp.fit(x_train, y_train)

In [ ]:
y_test_mlp = mlp.predict(x_test)
y_train_mlp = mlp.predict(x_train)

## Evaluation

In [ ]:
acc_train_mlp = accuracy_score(y_train,y_train_mlp)
acc_test_mlp = accuracy_score(y_test,y_test_mlp)

print("Multilayer Perceptrons: Accuracy on training Data: {:.3f}".format(acc_train_mlp))
print("Multilayer Perceptrons: Accuracy on test Data: {:.3f}".format(acc_test_mlp))

In [ ]:
storeResults('Multilayer Perceptrons', acc_train_mlp, acc_test_mlp)

# Keras Sequential

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
x.shape

In [ ]:
keras_sequential = tf.keras.models.Sequential()
keras_sequential.add(tf.keras.layers.Input(shape=19))
keras_sequential.add(tf.keras.layers.Dense(100))
keras_sequential.add(keras.layers.Activation('sigmoid'))
keras_sequential.add(tf.keras.layers.Dense(100))
keras_sequential.add(keras.layers.Activation('sigmoid'))
keras_sequential.add(tf.keras.layers.Dense(1, activation='sigmoid'))

loss = 'binary_crossentropy'
optim = 'nadam'
metrics = ["accuracy"]

keras_sequential.compile(loss=loss, optimizer=optim, metrics=metrics)

In [ ]:
keras_sequential.fit(x, y, epochs=500, shuffle=True, verbose=1, validation_split=0.2)

## Cross Validation


In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def built_clasifier():
  keras_sequential = tf.keras.models.Sequential()
  keras_sequential.add(tf.keras.layers.Input(shape=19))
  keras_sequential.add(tf.keras.layers.Dense(100))
  keras_sequential.add(keras.layers.Activation('sigmoid'))
  keras_sequential.add(tf.keras.layers.Dense(100))
  keras_sequential.add(keras.layers.Activation('sigmoid'))
  keras_sequential.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  loss = 'binary_crossentropy'
  optim = 'nadam'
  metrics = ["accuracy"]
  keras_sequential.compile(loss=loss, optimizer=optim, metrics=metrics)
  return keras_sequential

keras_sequential = KerasClassifier(build_fn = built_clasifier, batch_size = 100, epochs = 100)
accuracies = cross_val_score(estimator = keras_sequential, X = x_train, y = y_train, cv=10, n_jobs = -1)

In [ ]:
accuracies

## Model Test

In [ ]:
seq_train_auto = keras_sequential.evaluate(x_train, y_train, verbose=1, batch_size=1024)[1]
seq_test_auto = keras_sequential.evaluate(x_test, y_test, verbose=1, batch_size=1024)[1]

print('\nSequential: Accuracy on training Data: {:.3f}' .format(seq_train_auto))
print('Sequential: Accuracy on test Data: {:.3f}' .format(seq_test_auto))

In [ ]:
storeResults('KerasSequential', seq_train_auto, seq_test_auto)

In [ ]:
#score, acc = model.evaluate(x_test, y_test, verbose=1, batch_size=1024)
#print("Model Accuracy: {:0.2f}%".format(acc * 100))

In [ ]:
#tensorToPredict = tf.convert_to_tensor([[222,47,38,0,1]])
#tensorToPredict.shape

In [ ]:
#tensorToPredict = tf.convert_to_tensor([[222,1281,47,1,1]])
#tensorToPredict.dtype
#print(model.predict(tensorToPredict))

## Model Export

In [ ]:
keras_sequential.save("model.h5")
# tensorflowjs_converter --input_format keras lts.h5 lts

# Models compare

In [ ]:
results = pd.DataFrame({ 'ML Model': ML_Model,    
    'Train Accuracy': acc_train,
    'Test Accuracy': acc_test})
results

In [ ]:
results.sort_values(by=['Test Accuracy'], ascending=False)

# Confusion Matrix and Metrics


In [ ]:
import seaborn as sns

models = [
]

separationPoint = 0.5
group_names = ['TN','FP','FN','TP']

for title, model in models:
  y_test_pred = model.predict_proba(x_test)[:, 1] > separationPoint
  cf_matrix=confusion_matrix(y_test,y_test_pred)
  group_counts = ["{0:0.0f}".format(value) for value in
                  cf_matrix.flatten()]
  group_percentages = ["{0:.2%}".format(value) for value in
                       cf_matrix.flatten()/np.sum(cf_matrix)]
  labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
            zip(group_names,group_counts,group_percentages)]
  labels = np.asarray(labels).reshape(2,2)
  ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
  ax.set_title('Confusion Matrix - ' + title + '\n\n');
  ax.set_xlabel('\nPredicted Values')
  ax.set_ylabel('Actual Values ');
  ## Ticket labels - List must be in alphabetical order
  ax.xaxis.set_ticklabels(['False','True'])
  ax.yaxis.set_ticklabels(['False','True'])
  ## Display the visualization of the Confusion Matrix.
  plt.show()
  if int(group_counts[3]) > 0:
    if int(group_counts[1]) > 0:
      precision = int(group_counts[3]) / (int(group_counts[3]) + int(group_counts[1]))
      print("Precision: ", precision)
      if int(group_counts[2]) > 0:
        recall = int(group_counts[3]) / (int(group_counts[3]) + int(group_counts[2]))
        print("Recall: ", recall)
        fOne = 2 * (precision * recall) / (precision + recall)
        print("F1: ", fOne)

y_test_pred = sequential.predict(x_test) > separationPoint
cf_matrix=confusion_matrix(y_test,y_test_pred)

group_names = ['TN','FP','FN','TP']

group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)

ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

ax.set_title('Confusion Matrix - Keras Sequential\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()
precision = int(group_counts[3]) / (int(group_counts[3]) + int(group_counts[1]))
print("Precision: ", precision)
recall = int(group_counts[3]) / (int(group_counts[3]) + int(group_counts[2]))
print("Recall: ", recall)
fOne = 2 * (precision * recall) / (precision + recall)
print("F1: ", fOne)

# Time taken

In [ ]:
df_visited = loadDataset("timeTaken.csv")

In [ ]:
df_visited.hist(bins=50, figsize=(5, 5))